# Advanced Minimax Algorithm

The minimax algorithm developed for the tic-tac-toe game looked at all possible future moves and chose the one that would be most beneficial. This was a viable strategy because Tic Tac Toe is a small enough game that it wouldn't take too long to reach the leaves of the game tree.

However, games like Chess have much larger trees. There are 20 different options for the first move in Chess, compared to 9 in Tic-Tac-Toe. On top of that, the number of possible moves often increases as a chess game progresses. Traversing to the leaves of a Chess tree using a `for loop` and `recursion` would take too much computational power.

There are two techniques we can use to solve this problem. The first, puts a hard limit on how far down the tree you allow the algorithm to go. The second uses a technique called pruning to avoid exploring parts of the tree that we know will be useless.

### Limiting depth by stopping recursion early

Being able to stop before reaching the leaves is critically important for the efficiency of this algorithm. It could take literal days to reach the leaves of a game of chess. We'll just look a few moves ahead, limiting the algorithm's understanding of the game, but makeing the runtime realistic.

In order to implement this, we'll add another parameter to our function called depth. Every time we make a recursive call, we'll decrease depth by 1. We'll also have to edit our base case condition. We now want to stop if the game is over (we've hit a leaf), or if depth is 0.

```py
def minimax(input_board, minimizing_player, depth):
  # Base Case
  if game_is over(input_bopard):
    return ...
  else:
    # …
    # Recursive Call
    hypothetical_value = minimax(new_board, True, depth - 1)[0]
```

```py
# current version
from connect_four import *
import random
random.seed(108)

new_board = make_board()

# Add a third parameter named depth
def minimax(input_board, is_maximizing, depth):
  # Change this if statement to also check to see if depth = 0
  if game_is_over(input_board) or depth == 0:
    return [evaluate_board(input_board), ""]
  best_move = ""
  if is_maximizing == True:
    best_value = -float("Inf")
    symbol = "X"
  else:
    best_value = float("Inf")
    symbol = "O"
  for move in available_moves(input_board):
    new_board = deepcopy(input_board)
    select_space(new_board, move, symbol)
    #Add a third parameter to this recursive call
    hypothetical_value = minimax(new_board, not is_maximizing, depth - 1)[0]
    if is_maximizing == True and hypothetical_value > best_value:
      best_value = hypothetical_value
      best_move = move
    if is_maximizing == False and hypothetical_value < best_value:
      best_value = hypothetical_value
      best_move = move
  return [best_value, best_move]

print(minimax(new_board, True, 3))
```

By adding the depth parameter to our function, we've prevented it from spending days trying to reach the end of the tree. But we still have a problem: our evaluation function doesn't know what to do if we're not at a leaf. Right now, we're returning positive infinity if Player 1 has won, negative infinity if Player 2 has won, and 0 otherwise. Unfortunately, since we're not making it to the leaves of the board, neither player has won and we're always returning 0. We need to rewrite our evaluation function.

Writing an evaluation function takes knowledge about the game you're playing. It is the part of the code where a programmer can infuse their creativity into their AI. If you're playing Chess, your evaluation function could count the difference between the number of pieces each player owns. For example, if white had 15 pieces, and black had 10 pieces, the evaluation function would return 5. This evaluation function would make an AI that prioritizes capturing pieces above all else.

You could go even further — some pieces might be more valuable than others. Your evaluation function could keep track of the value of each piece to see who is ahead. This evaluation function would create an AI that might really try to protect their queen. You could even start finding more abstract ways to value a game state. Maybe the position of each piece on a Chess board tells you something about who is winning.

It's up to you to define what you value in a game. These evaluation functions could be incredibly complex. If the maximizing player is winning (by your definition of what it means to be winning), then the evaluation function should return something greater than 0. If the minimizing player is winning, then the evaluation function should return something less than 0.

```py
def evaluate_board(board):
    if has_won(board, "X"):
      return float("Inf")
    elif has_won(board, "O"):
      return -float("Inf")
    # how to evaulate a board when no one has one
    else:
      num_top_x = 0
      num_top_o = 0
      # looking at every square, in every column going from top to bottom
      for column in board:
        for square in column:
          if square == 'X':
            num_top_x += 1
            break
          if square == 'O':
            num_top_o += 1
            break

      return num_top_x - num_top_o
```

### Alpha-Beta Pruning

In order to traverse farther down the tree without dramatically increasing the runtime, we can implement a technique called alpha-beta pruning.

The core idea behind alpha-beta pruning is to ignore parts of the tree that we know will be dead ends. For example, consider the gif on this page. When determining the "value" of the node labeled E, we can stop looking at possible moves as soon as it discovers the 8 node.

![alppha-beta-pruning](img/alphabetapruning.gif)

We know that B will only consider values less than or equal to 5, and E will only consider values greater than or equal to 8. We know that node B will never care about E's value and we can stop looking through E's moves.

![minimax](img/minimax-2.png)

Similarly, we can prune a large section from the right half of the tree. There comes a point where node A will only consider values greater than or equal to 5 and node C will only consider values 3 and below. We can stop looking at all of the C's children because we know they will never be relevant.

### Implementing Alpha-Beta Pruning

Alpha-beta pruning is accomplished by keeping track of two variables for each node — `alpha` and `beta`. 

`alpha` keeps track of the **minimum** score the **maximizing** player can possibly get. It starts at negative infinity and gets updated as that minimum score increases.

`beta` represents the **maximum** score the **minimizing** player can possibly get. It starts at positive infinity and will decrease as that maximum possible score decreases.

For any node, if `alpha` is **greater than or equal** to `beta`, that means that we can **stop** looking through that node's children.

To implement this in our code, we'll have to include two new parameters in our function — `alpha` and `beta`. When we first call `minimax()` we'll set `alpha` to **negative infinity** and `beta` to **positive infinity**.

We also want to make sure we pass alpha and beta into our recursive calls. We're passing these two values down the tree.

Next, we want to check to see if we should reset alpha and beta. In the maximizing case, we want to reset alpha if the newly found best_value is greater than alpha. In the minimizing case, we want to reset beta if best_value is less than beta.

Finally, after resetting alpha and beta, we want to check to see if we can prune. If alpha is greater than or equal to beta, we can break and stop looking through the other potential moves.

```py
def minimax(input_board, is_maximizing, depth, alpha=-float('Inf'), beta=float('Inf')):
  # Base case - the game is over, so we return the value of the board
  if game_is_over(input_board) or depth == 0:
    return [evaluate_board(input_board), "", alpha, beta]
  best_move = ""
  if is_maximizing == True:
    best_value = -float("Inf")
    symbol = "X"
  else:
    best_value = float("Inf")
    symbol = "O"
  for move in available_moves(input_board):
    new_board = deepcopy(input_board)
    select_space(new_board, move, symbol)
    hypothetical_value = minimax(new_board, not is_maximizing, depth - 1, alpha, beta)[0]
    if is_maximizing == True and hypothetical_value > best_value:
      best_value = hypothetical_value
      alpha = max(alpha, best_value) # return the max value
      best_move = move
    if is_maximizing == False and hypothetical_value < best_value:
      best_value = hypothetical_value
      beta = min(beta, best_value)
      best_move = move
    # stop looking through the remaining possible moves of the current game state.  
    if alpha >= beta:
      break
  return [best_value, best_move, alpha, beta]
```  

### Review

We've edited the `minimax()` function to work with games that are more complicated than Tic Tac Toe. The core of the algorithm is identical, but we've added two major improvements:

We wrote an evaluation function specific to our understanding of the game (in this case, Connect Four). This evaluation function allows us to stop the recursion before reaching the leaves of the game tree.

We implemented alpha-beta pruning. By cleverly detecting useless sections of the game tree, we're able to ignore those sections and therefore look farther down the tree.